# What is about 

Target 'pdgfr_inhibitor' is among few other targets where boostings perform quite well (that is result of some previous analysis). 

Here we demonstate that comparing with logreg, as well as show how can it be improved further by blending over different folds

For LogReg we use C specially optimized for that target (again result of previous analysis). 

LogReg      :  0.02581

LGB         :  0.02478 

LGB Blend 20:  0.02360

LR + LGB    :  0.022392 (just one LR and one LGB, if one LR and many LGB - that worsens results). 

probably may be improved futher combining several blends of logregs and several blends of boostings



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns

import time
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df0 = df.copy()
df['cp_type'] = df['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':1.0}) # Forget about control group  
df['cp_dose'] = df['cp_dose'].map({'D1':0.0, 'D2':1.0})
df['cp_time'] = df['cp_time'].map({24:0.0, 48: .5 , 72:1.0})
X = df.copy()
X_save = X.copy()
df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col = 0)
df0_test = df_test.copy()
df_test['cp_type'] = df_test['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
df_test['cp_dose'] = df_test['cp_dose'].map({'D1':0.0, 'D2':1.0})
df_test['cp_time'] = df_test['cp_time'].map({24:0.0, 48: .5 , 72:1.0})

y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col = 0 )
y_save = y.copy()
print(y.iloc[:3,:2])
df

In [ ]:
y_save.sum(axis = 0 ).sort_values(ascending = False)

In [ ]:
#import warnings
#warnings.filterwarnings("ignore")

In [ ]:
# LGB Boosting params 
# Params chosen as the best for THAT PARTICULAR target among 6 available public notebooks params

params = {
'n_estimators': 50,
'objective': 'binary',
'boosting_type': 'gbdt',
'max_depth': 3,
'learning_rate': 0.08,
'subsample': 0.72,
'subsample_freq': 4,
'feature_fraction': 0.4,
'lambda_l1': 1,
'lambda_l2': 1,
'seed': 217, # SEED,
'early_stopping_rounds': 40,
}



In [ ]:
y = y_save['pdgfr_inhibitor']
y.sum(),log_loss(y,np.ones_like(y)*y.mean())

In [ ]:
X.shape, y.shape

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Logistic Regression for benchmark

In [ ]:
#for C_effective in C*np.linspace(0.5,1,10): # np.array([0.5,1, 2]):
C = 0.02
model = LogisticRegression( C = C, max_iter = 500 ) #, penalty='l1', solver = 'liblinear' )  
cv_score = cross_val_score(model, X,y,cv = 3, scoring= 'neg_log_loss')
np.mean(cv_score), np.std(cv_score), cv_score

In [ ]:
#for C_effective in C*np.linspace(0.5,1,10): # np.array([0.5,1, 2]):
C = 0.02
model = LogisticRegression( C = C ) #, penalty='l1', solver = 'liblinear' )  
cv_score = cross_val_score(model, X,y,cv = 3, scoring= 'neg_log_loss')
mp.mean(cv_score), np.std(cv_score), cv_score

In [ ]:
np.mean([-0.02614263, -0.02758013, -0.02370765])

# Core - blend several boosting made out different splits to folds 

In [ ]:
n_blends = 20

target_name = 'pdgfr_inhibitor' 
y = y_save[target_name]

rs = 0
n_splits = 3

cv_score = []
t0 = time.time() ; t00 = time.time()
oof = np.zeros(X.shape[0])

df_stat = pd.DataFrame()
cnt4df_stat = 0

#y_pred_submit = np.zeros( len(df_test) )
cnt_blend_submit = 0
y_pred_oof_blend = np.zeros_like(y,dtype = float)
cnt_blend_oof = 0
for cnt in range(n_blends):
    rs =  np.random.randint(10**7)
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )
    y_pred_oof = np.zeros_like(y,dtype = float)
    list_loss_train = []
    for n, (train_idx, valid_idx) in enumerate(skf.split(X, y) ):
        # train test split
        X_train_train = X.iloc[train_idx]
        y_train_train = y.iloc[train_idx]
        X_train_valid = X.iloc[valid_idx]
        y_train_valid = y.iloc[valid_idx]

        # lgb dataset
        lgb_train = lgb.Dataset(data=X_train_train, label=y_train_train)
        lgb_valid = lgb.Dataset(data=X_train_valid, label=y_train_valid)

        # fit
        model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=0)
        #fi['importance'] += model.feature_importance(importance_type="gain") / N_SPLITS
        #print(model)
        # save mod?p(model, open(f'model_{random_state}_{n}_{target}.pkl', 'wb'))                

        # predict
        y_pred_oof[valid_idx] = model.predict(X_train_valid, num_iteration=model.best_iteration)
        #y_preds += model.predict(test[features]) / N_SPLITS
        #print(model.best_iteration) 

        #list_loss_train.append( log_loss(y_train_train,  model.predict_proba(X_train)[:,1]) )
        list_loss_train.append( log_loss(y_train_train,  model.predict(X_train_train, num_iteration=model.best_iteration)) )
        
        cv_score.append( -log_loss(y_train_valid, y_pred_oof[valid_idx] ) )
        
    y_pred_oof_blend = (y_pred_oof_blend*cnt_blend_oof + y_pred_oof) / (cnt_blend_oof + 1) # blend
    cnt_blend_oof += 1

    df_stat.loc[cnt4df_stat,'Target'] = target_name
    df_stat.loc[cnt4df_stat,'Count Blend'] = cnt_blend_oof
    df_stat.loc[cnt4df_stat,'LogLoss OOF'] = -log_loss(y, y_pred_oof )
    df_stat.loc[cnt4df_stat,'LogLoss Blend OOF'] = -log_loss(y, y_pred_oof_blend )
    df_stat.loc[cnt4df_stat,'LogLoss Train'] = -np.mean( list_loss_train )
    df_stat.loc[cnt4df_stat,'Seconds Passed'] = np.round( time.time() - t00 )
    df_stat.loc[cnt4df_stat,'Random Seed'] = rs
    #df_stat.loc[cnt4df_stat,'C'] = C
    cnt4df_stat += 1
df_stat

In [ ]:
# prepare statistics to make plots/etc

lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )

name_oof = lc[0]
df_stat2 = pd.DataFrame()
for i,c in enumerate(y_save.columns):
    if c not in list(  df_stat['Target'] ) : continue
    
    m = df_stat['Target'] == c
    df_stat2.loc[c,'Internal Numero'] = i 
    df_stat2.loc[c,'Target Sum'] = y_save[c].sum()
    df_stat2.loc[c,'Blend gain * 1e5'] = np.round( - 1e5*( df_stat[m]['LogLoss Blend OOF'].iat[-1] - df_stat[m]['LogLoss OOF'].mean() ) , 1)
    
    df_stat2.loc[c,'Logloss predict by mean'] = log_loss(y_save[c], np.ones_like(y_save[c])*y_save[c].mean() ) 
    df_stat2.loc[c,'Logloss OOF'] = df_stat[m]['LogLoss OOF'].mean()
    df_stat2.loc[c,'Logloss Blend OOF'] = df_stat[m]['LogLoss Blend OOF'].iat[-1]
    df_stat2.loc[c, 'LogLoss Train'] = df_stat[m][ 'LogLoss Train'].mean()
    
    df_stat2.loc[c,'Logloss Std'] = df_stat[m]['LogLoss OOF'].std()
    df_stat2.loc[c,'Logloss Train Std'] = df_stat[m]['LogLoss Train'].std()
    #df_stat[m]
df_stat2['Internal Numero'] = df_stat2['Internal Numero'].astype(int)    

df_stat2.to_csv('df_stat2_by_targets_aggregated.csv')

df_stat2.sort_values('Blend gain * 1e5', ascending = False)

# Main plots - illustration how blend works

In [ ]:
lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )
d = df_stat.groupby('Count Blend')[lc].mean()
fig = plt.figure(figsize = (15,6))
fig.add_subplot(1,2,1)
for c in d.columns:
    if 'Train' not in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
fig.add_subplot(1,2,2)
for c in d.columns:
    if 'Train' in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
plt.show()
#print( d[lc].mean() )
#print(d.tail(1) )
print('No blend:', np.round( d['LogLoss OOF'].mean() , 5 )  , 'Blend', np.round( d.tail(1)['LogLoss Blend OOF'].iat[0], 5) )

blend_gain = - (  d['LogLoss OOF'].mean() - d.tail(1)['LogLoss Blend OOF'].iat[0] )
print('Blend gain in 5-th digits:', np.round( 1e5* blend_gain , 0)  )

# Saved result for 10 blends

In [ ]:
lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )
d = df_stat.groupby('Count Blend')[lc].mean()
fig = plt.figure(figsize = (15,6))
fig.add_subplot(1,2,1)
for c in d.columns:
    if 'Train' not in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
fig.add_subplot(1,2,2)
for c in d.columns:
    if 'Train' in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
plt.show()
#print( d[lc].mean() )
#print(d.tail(1) )
print('No blend:', np.round( d['LogLoss OOF'].mean() , 5 )  , 'Blend', np.round( d.tail(1)['LogLoss Blend OOF'].iat[0], 5) )

blend_gain = - (  d['LogLoss OOF'].mean() - d.tail(1)['LogLoss Blend OOF'].iat[0] )
print('Blend gain in 5-th digits:', np.round( 1e5* blend_gain , 0)  )

# Blend one LogReg with many boostings - best is just blend one boostings

probably may be improved futher combining several blends of logregs and several blends of boostings



In [ ]:
n_blends = 20


target_name = 'pdgfr_inhibitor' 
y = y_save[target_name]

y_pred_oof_blend = np.zeros_like(y,dtype = float)
cnt_blend_oof = 0


# First blend - log reg

C = 0.02
model = LogisticRegression( C = C ) #, penalty='l1', solver = 'liblinear' )  
y_pred_oof_blend = cross_val_predict(model, X,y,cv = 3,  method = 'predict_proba')[:,1] #
cv_score = -log_loss(y, y_pred_oof_blend ) # , scoring= 'neg_log_loss')
print('LR Logloss:', cv_score )

cnt_blend_oof = 1




n_splits = 3
cv_score = []
t0 = time.time() ; t00 = time.time()

df_stat = pd.DataFrame()
cnt4df_stat = 0

#y_pred_submit = np.zeros( len(df_test) )
cnt_blend_submit = 0
for cnt in range(n_blends):
    rs =  np.random.randint(10**7)
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )
    y_pred_oof = np.zeros_like(y,dtype = float)
    list_loss_train = []
    for n, (train_idx, valid_idx) in enumerate(skf.split(X, y) ):
        # train test split
        X_train_train = X.iloc[train_idx]
        y_train_train = y.iloc[train_idx]
        X_train_valid = X.iloc[valid_idx]
        y_train_valid = y.iloc[valid_idx]

        # lgb dataset
        lgb_train = lgb.Dataset(data=X_train_train, label=y_train_train)
        lgb_valid = lgb.Dataset(data=X_train_valid, label=y_train_valid)

        # fit
        model = lgb.train(params, lgb_train, valid_sets=lgb_valid, verbose_eval=0)
        #fi['importance'] += model.feature_importance(importance_type="gain") / N_SPLITS
        #print(model)
        # save mod?p(model, open(f'model_{random_state}_{n}_{target}.pkl', 'wb'))                

        # predict
        y_pred_oof[valid_idx] = model.predict(X_train_valid, num_iteration=model.best_iteration)
        #y_preds += model.predict(test[features]) / N_SPLITS
        #print(model.best_iteration) 

        #list_loss_train.append( log_loss(y_train_train,  model.predict_proba(X_train)[:,1]) )
        list_loss_train.append( log_loss(y_train_train,  model.predict(X_train_train, num_iteration=model.best_iteration)) )
        
        cv_score.append( -log_loss(y_train_valid, y_pred_oof[valid_idx] ) )
        
    y_pred_oof_blend = (y_pred_oof_blend*cnt_blend_oof + y_pred_oof) / (cnt_blend_oof + 1) # blend
    cnt_blend_oof += 1

    df_stat.loc[cnt4df_stat,'Target'] = target_name
    df_stat.loc[cnt4df_stat,'Count Blend'] = cnt_blend_oof
    df_stat.loc[cnt4df_stat,'LogLoss OOF'] = -log_loss(y, y_pred_oof )
    df_stat.loc[cnt4df_stat,'LogLoss Blend OOF'] = -log_loss(y, y_pred_oof_blend )
    df_stat.loc[cnt4df_stat,'LogLoss Train'] = -np.mean( list_loss_train )
    df_stat.loc[cnt4df_stat,'Seconds Passed'] = np.round( time.time() - t00 )
    df_stat.loc[cnt4df_stat,'Random Seed'] = rs
    #df_stat.loc[cnt4df_stat,'C'] = C
    cnt4df_stat += 1
df_stat

In [ ]:
# prepare statistics to make plots/etc

lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )

name_oof = lc[0]
df_stat2 = pd.DataFrame()
for i,c in enumerate(y_save.columns):
    if c not in list(  df_stat['Target'] ) : continue
    
    m = df_stat['Target'] == c
    df_stat2.loc[c,'Internal Numero'] = i 
    df_stat2.loc[c,'Target Sum'] = y_save[c].sum()
    df_stat2.loc[c,'Blend gain * 1e5'] = np.round( - 1e5*( df_stat[m]['LogLoss Blend OOF'].iat[-1] - df_stat[m]['LogLoss OOF'].mean() ) , 1)
    
    df_stat2.loc[c,'Logloss predict by mean'] = log_loss(y_save[c], np.ones_like(y_save[c])*y_save[c].mean() ) 
    df_stat2.loc[c,'Logloss OOF'] = df_stat[m]['LogLoss OOF'].mean()
    df_stat2.loc[c,'Logloss Blend OOF'] = df_stat[m]['LogLoss Blend OOF'].iat[-1]
    df_stat2.loc[c, 'LogLoss Train'] = df_stat[m][ 'LogLoss Train'].mean()
    
    df_stat2.loc[c,'Logloss Std'] = df_stat[m]['LogLoss OOF'].std()
    df_stat2.loc[c,'Logloss Train Std'] = df_stat[m]['LogLoss Train'].std()
    #df_stat[m]
df_stat2['Internal Numero'] = df_stat2['Internal Numero'].astype(int)    

df_stat2.to_csv('df_stat2_by_targets_aggregated.csv')

df_stat2.sort_values('Blend gain * 1e5', ascending = False)

In [ ]:
lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )
d = df_stat.groupby('Count Blend')[lc].mean()
fig = plt.figure(figsize = (15,6))
fig.add_subplot(1,2,1)
for c in d.columns:
    if 'Train' not in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
fig.add_subplot(1,2,2)
for c in d.columns:
    if 'Train' in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
plt.show()
#print( d[lc].mean() )
#print(d.tail(1) )
print('No blend:', np.round( d['LogLoss OOF'].mean() , 5 )  , 'Blend', np.round( d.tail(1)['LogLoss Blend OOF'].iat[0], 5) )

blend_gain = - (  d['LogLoss OOF'].mean() - d.tail(1)['LogLoss Blend OOF'].iat[0] )
print('Blend gain in 5-th digits:', np.round( 1e5* blend_gain , 0)  )